# Data Work In Progress

NB: NOT the report but a place to brain dump analysis, plots etc etc - Then copy relevnt code and results to the report

## Actions
- compare the CSV files to the extracted JSON
- is the data structure ok?



In [1]:
# General setup and imports used throughout the remaining cells
#

# Libraries For file handling and dataframes
import os
import json
from IPython.display import display
import pandas as pd 

# Libraries ....
## cccccc

# A simple utility function to obtain and summarise key elements of a given dataframe
def print_file_summary(data_frame):
    # Create a temporary df and ensure no lists remain, so that unique items can be identified for uniquness
    temp_df = data_frame.copy()
    temp_df = temp_df.map(lambda cell: str(cell) if isinstance(cell, list) else cell)
    
    # Calculate some 
    summary_of_df = pd.DataFrame({'Count': data_frame.count(),
                                 'Missing': data_frame.isnull().sum(), 'Empty': 0,
                                 'Unique': temp_df.nunique(),
                                 'Type': data_frame.dtypes, 
                                 'String': 0, 'Int': 0, 'Float': 0, 'List': 0
                                 })
    summary_of_df['Empty'] = (data_frame == '').sum()
    summary_of_df['String'] = data_frame.map(lambda cell: isinstance(cell, str)).sum()
    summary_of_df['Int'] = data_frame.map(lambda cell: isinstance(cell, int)).sum()
    summary_of_df['Float'] = data_frame.map(lambda cell: isinstance(cell, float)).sum()
    summary_of_df['List'] = data_frame.map(lambda cell: isinstance(cell, list)).sum()

    display(summary_of_df)


# Data Exploration & Validation

In [2]:
# Load the JSON product file and examine the format and content
# NB: Use pandas json load to directly create a dataframe

# File handling and dataframes - from the designated data directory within the current working directory
DATA_DIRECTORY = 'JCPenney_Data_Original'

# Products file
file_name = 'jcpenney_products.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')

source_jcp_products_df = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(source_jcp_products_df)
print(f'First 3 Rows')
display(source_jcp_products_df.head(3))


File Summary for: jcpenney_products.json


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
uniq_id,7982,0,0,7982,object,7982,0,0,0
sku,7982,0,67,6044,object,7982,0,0,0
name_title,7982,0,0,6002,object,7982,0,0,0
description,7982,0,543,5620,object,7982,0,0,0
list_price,7982,0,2166,1037,object,7982,0,0,0
sale_price,7982,0,18,2063,object,7982,0,0,0
category,7982,0,636,1169,object,7982,0,0,0
category_tree,7982,0,636,1997,object,7982,0,0,0
average_product_rating,7982,0,0,153,float64,0,0,7982,0
product_url,7982,0,0,7982,object,7982,0,0,0


First 3 Rows


,uniq_id,sku,name_title,description,list_price,sale_price,category,category_tree,average_product_rating,product_url,product_image_urls,brand,total_number_reviews,Reviews,Bought With
0,b6c0b6bea69c722939585baeac73c13d,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'fsdv4141', 'Review': 'You never hav...","[898e42fe937a33e8ce5e900ca7a4d924, 8c02c262567..."
1,93e5272c51d8cce02597e3ce67b7ad0a,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,3.000,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'tpcu2211', 'Review': 'You never hav...","[bc9ab3406dcaa84a123b9da862e6367d, 18eb69e8fc2..."
2,013e320f2f2ec0cf5b3ff5418d688528,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,view all,jcpenney|women|view all,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'pcfg3234', 'Review': 'You never hav...","[3ce70f519a9cfdd85cdbdecd358e5347, b0295c96d2b..."


In [ ]:
# For reviews create a seperate dataframe and remove from products dataframe
#

jcp_user_reviews_df = pd.DataFrame()
for next_row in source_jcp_products_df.itertuples(index=False):
    temp_reviews = next_row.Reviews
    #print(f'UI: {next_row.uniq_id}, {temp_reviews}, {type(temp_reviews)}')
    temp_dict_string = json.dumps(temp_reviews)
    temp_reviews_df = pd.DataFrame(json.loads(temp_dict_string))
    temp_reviews_df.insert(0, 'uniq_id', next_row.uniq_id)
    #print(temp_reviews_df)
    jcp_user_reviews_df = pd.concat([jcp_user_reviews_df, temp_reviews_df])

# Tidy up and create products file without reviews
del temp_reviews
del temp_dict_string
del temp_reviews_df
jcp_products_df = source_jcp_products_df.drop('Reviews', axis=1)

# TODO: Same for bought with?
# TODO: Set index for products DF?
# TODO: cross-checks eg average score?
# TODO: column-names all to lower case, change ints and strings?
# TODO: cross-check user names to user names file?

In [ ]:
# Load the JSON reviewers file and examine the format and content
# NB: Use pandas json load to directly create a dataframe

# File handling and dataframes - from the designated data directory within the current working directory
DATA_DIRECTORY = 'JCPenney_Data_Original'

# Products file
file_name = 'jcpenney_reviewers.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')
df_jcp_reviewers = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(df_jcp_reviewers)
print(f'First 3 Rows')
display(df_jcp_reviewers.head(3))

File Summary for: jcpenney_reviewers.json


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
Username,5000,0,0,4999,object,5000,0,0,0
DOB,5000,0,0,52,object,5000,0,0,0
State,5000,0,0,57,object,5000,0,0,0
Reviewed,5000,0,0,4030,object,0,0,0,5000


First 3 Rows


,Username,DOB,State,Reviewed
0,bkpn1412,31.07.1983,Oregon,[cea76118f6a9110a893de2b7654319c0]
1,gqjs4414,27.07.1998,Massachusetts,[fa04fe6c0dd5189f54fe600838da43d3]
2,eehe1434,08.08.1950,Idaho,[]


# Data Sourcing & Load

- Sources used, where from, type
- How up to date, when published
- Here show use of: csv, JSON, requests ... Pandas

### Source Datasets from JC Penny



#### Products
- products.csv
- products.json

?? Same data content but in .csv and .json format
?? both loaded and checked to confirm same content

Data

In [ ]:
# Load the JSON files and examine the format and content
# NB: Use pandas json load to directly create a dataframe, rather then importing and using the JSON library

# File handling and dataframes - from the designated data directory within the current working directory
DATA_DIRECTORY = 'JCPenney_Data_Original'

# Products file
file_name = 'jcpenney_products.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')
df_jcp_products = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(df_jcp_products)

# Reviewers file
file_name = 'jcpenney_reviewers.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')
df_jcp_products = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(df_jcp_products)


In [ ]:
# Load the CSV files and examine the format and content
# Compare with the JSON files

In [ ]:
# Load the five JC Penny datasets - in .csv and .hson format
# Load the .csv and .json formats 

# File handling and dataframes - from the designated data directory within the current working directory
# Do not amend the original source file, but retain until cleaned dataframes are produced for working
import os
import pandas as pd 
DATA_DIRECTORY = 'JCPenney_Data_Original'

# Load the products .csv and .json file, exit if do not exist or are invalid
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, 'products.csv')
if not os.path.isfile(file_path):
    raise Exception(f"File not found: {file_path}")
df_jcp_products = pd.read_csv(file_path)

# TODO: Compare the csv and json files ... the same?

# Initial look at the format of both files and data fields contained

print('Products File - Summary')
summary_of_df = pd.DataFrame({'Count': df_jcp_products.count(),
                              'Missing': df_jcp_products.isnull().sum() })
print(summary_of_df)
print(f'\nProducts File - Columns and First 3 Rows')
print(df_jcp_products.head(3))


## Observations from Data Load

- JSON has more data than CSV, and has linked the products and review into one file. So sue this

- Duplicate username


### TODO
- But still load CSV just to demonstrate why not using
- some missing to tidy up
- Look at unique count
- rename uniqu-id to reenforce it is the review id
- explain SKU and all the fields?
- Go through one by one yo understand structures eg the nested Reviews and Bought With fields
- Use an image lookup URL to retrieve and print an image
- Product URLs don't work


# Data Exploration

- What is the structure, fields, meanings
- What are the relationships

# Data Cleaning

- find problems and resolve
- missing, null
- duplicates
- outliers, anomolies


# Potential Items / Observations to Investigate Further

